In [46]:
import pandas as pd
import numpy as np
import animalhelper as ah

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
df = pd.read_csv('./data/engineered.csv', index_col = 0, dtype={'intake_condition': str})

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67211 entries, 0 to 28302
Data columns (total 16 columns):
age                 67211 non-null int64
animal_id           67211 non-null object
animal_type         67211 non-null object
datetime_in         67211 non-null object
days_in_shelter     67211 non-null int64
dow                 67211 non-null int64
fixed_status        67211 non-null object
gender              67211 non-null int64
group               67211 non-null object
intake_condition    67211 non-null object
intake_season       67211 non-null object
intake_type         67211 non-null object
mix                 67211 non-null int64
name                67211 non-null int64
outcome_type        67211 non-null object
simple_color        67211 non-null object
dtypes: int64(6), object(10)
memory usage: 8.7+ MB


In [79]:
df_mod = pd.DataFrame()

In [80]:
df_mod[['Cat', 'Dog']] = pd.get_dummies(df['animal_type'])

In [81]:
df_mod['age'] = df['age']

In [82]:
df['fixed_status'].value_counts()

intact     45760
fixed      18644
unknown     2807
Name: fixed_status, dtype: int64

## Impute unknown fixed status values

In [83]:
df['fixed_status'] = df['fixed_status'].apply(lambda x: ah.impute_fixed_status(x))

In [84]:
df_mod[['fixed', 'intact']] = pd.get_dummies(df['fixed_status'])

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67211 entries, 0 to 28302
Data columns (total 16 columns):
age                 67211 non-null int64
animal_id           67211 non-null object
animal_type         67211 non-null object
datetime_in         67211 non-null object
days_in_shelter     67211 non-null int64
dow                 67211 non-null int64
fixed_status        67211 non-null object
gender              67211 non-null int64
group               67211 non-null object
intake_condition    67211 non-null object
intake_season       67211 non-null object
intake_type         67211 non-null object
mix                 67211 non-null int64
name                67211 non-null int64
outcome_type        67211 non-null object
simple_color        67211 non-null object
dtypes: int64(6), object(10)
memory usage: 8.7+ MB


In [86]:
df_mod[['male', 'female']] = pd.get_dummies(df['gender'])

# Clean up groups quickly

In [87]:
df['group'].value_counts()

short hair       27058
Terrier           9524
Toy               7376
Sporting          6530
Herding           5897
Working           4604
Hound             2995
Non-Sporting      1870
long hair         1245
Terrier & Toy      111
Unknown              1
Name: group, dtype: int64

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67211 entries, 0 to 28302
Data columns (total 16 columns):
age                 67211 non-null int64
animal_id           67211 non-null object
animal_type         67211 non-null object
datetime_in         67211 non-null object
days_in_shelter     67211 non-null int64
dow                 67211 non-null int64
fixed_status        67211 non-null object
gender              67211 non-null int64
group               67211 non-null object
intake_condition    67211 non-null object
intake_season       67211 non-null object
intake_type         67211 non-null object
mix                 67211 non-null int64
name                67211 non-null int64
outcome_type        67211 non-null object
simple_color        67211 non-null object
dtypes: int64(6), object(10)
memory usage: 8.7+ MB


In [89]:
def clean_groups(s):
    if s == 'Terrier & Toy':
        return 'Terrier'
    return s

In [90]:
df['group'] = df['group'].apply(lambda x: clean_groups(x))

In [91]:
df = df[~(df['group'] == 'Unknown')]

In [92]:
df['group'].value_counts()

short hair      27058
Terrier          9635
Toy              7376
Sporting         6530
Herding          5897
Working          4604
Hound            2995
Non-Sporting     1870
long hair        1245
Name: group, dtype: int64